In [42]:
import time
import sched
from datetime import datetime, tzinfo
from elasticsearch import Elasticsearch
#matplotlib.rc('font', **{'size': 15})
from elasticsearch.helpers import scan

import json
import redis

import requests


In [43]:
#rs = redis.Redis('aianalytics13.cern.ch')
#tmp = rs.get('metrics#UNI-DORTMUND:BNL-ATLAS')
#out = json.loads(tmp)

In [44]:



url = 'http://dashb-atlas-ddm.cern.ch/dashboard/request.py/matrix.json?activity=Data+Brokering&activity=Data+Consolidation&activity=Data+Export+Test&activity=Debug&activity=Deletion&activity=Express&activity=Functional+Test&activity=Group+Subscriptions&activity=Production&activity=Production+Input&activity=Production+Output&activity=Recovery&activity=Staging&activity=T0+Export&activity=T0+Tape&activity=User+Subscriptions&activity=default&activity=on&activity=on&activity=rucio-integration&activity=test&activity=test%3AT0_T1+export&activity=test%3AT1_T2+export&activity=testactivity10&activity=testactivity20&activity=testactivity70&activity_default_exclude=Upload%2FDownload+%28Job%29&activity_default_exclude=Upload%2FDownload+%28User%29&activity_default_exclude=Analysis+Download&activity_default_exclude=Analysis+Upload&activity_default_exclude=Production+Download&activity_default_exclude=Production+Upload&activity_default_exclude=CLI+Download&activity_default_exclude=CLI+Upload&src_grouping=site&src_grouping=token&dst_grouping=site&dst_grouping=token&interval=60'
res = requests.get(url).json()['transfers']['rows']
print (len(res))
#print(res)


#Mario: The ddm2 dashboard is filled with exactly these values, which come from a fancy pipeline out of DDM.

2918


In [45]:
#define scheduler
s = sched.scheduler(time.time, time.sleep)

#define variables
interval = 6000*10 #update every [interval] seconds

verbose = True

pattern = '%Y-%m-%d %H:%M:%S'



#debug
if verbose: print('Local time Chicago: ',datetime.now().strftime(pattern))

#number of successful transfers in interval
transfers = 0
#number of failed transfers
failures = 0

Local time Chicago:  2017-05-04 04:42:32


In [46]:
def process_last():
    print()
    #reset counters
    transfers = 0
    failures = 0
    now_datetime = datetime.now().strftime(pattern)
    if verbose: print('The clock is: ',now_datetime)
    if verbose: print('Epoch time: ',int(time.time())*1000)

    now = int(time.time()) * 1000
    
    past = now - interval*1000 #convert interval to epoch milliseconds
   
    
    
    query_done = {
        "size": 0,
        "query": {
        "bool": {
        "must": [
        {
            "range": {
            "@timestamp": {
                "gte": past,
                "lte": now,
                "format": "epoch_millis"
                }
            }
        },
            {"term": {"event_type": "transfer-done"}},
            {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
        }
    }
    query_failed = {
        "size": 0,
        "query": {
        "bool": {
        "must": [
        {
            "range": {
            "@timestamp": {
                "gte": past,
                "lte": now,
                "format": "epoch_millis"
                 }
            }
        },
            {"term": {"event_type": "transfer-failed"}},
            {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
        }
    }


    
    # make scrolls
    es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) #chicago elasticsearch
    #es = Elasticsearch(['es-atlas.cern.ch:9203'],timeout=60) #cern elasticsearch
    
    my_index = "atlas_rucio-events-2017*"
    scroll_done = scan(es, query=query_done, index=my_index, scroll='5m', timeout="5m", size=100)
    scroll_errors = scan(es, query=query_failed, index=my_index, scroll='5m', timeout="5m", size=100)
    
    print(scroll_done)
    
    for entry in scroll_done:
        transfers += 1
    
    for entry in scroll_errors:
        failures += 1
        
        
    if verbose: print('Number of successful transfers last ',interval,' seconds: ',transfers)
    if verbose: print('Number of failed transfers last ',interval,' seconds: ',failures)

    #schedule rerun of job in [interval] seconds
    print()
    s.enter(interval, 1, process_last, ())

In [47]:
process_last()
#s.run()


The clock is:  2017-05-04 04:42:32
Epoch time:  1493890952000
<generator object scan at 0x7f2f0c4ebc50>
Number of successful transfers last  60000  seconds:  0
Number of failed transfers last  60000  seconds:  0

